# load json file

In [197]:
def read_json_file(file):
    with open(file, "r") as r:
        response = r.read()
        response = response.replace('\n', '')
        response = response.replace('}{', '},{')
        response = "[" + response + "]"
        return json.loads(response)

In [198]:
Receipts = read_json_file('receipts.json')
Brand = read_json_file('brands.json')
Users = read_json_file('users.json')

In [199]:
Brand = pd.json_normalize(Brand)
Users = pd.json_normalize(Users)
Receipts = pd.json_normalize(Receipts)

In [298]:
# flatten the nested list in Receipts table, use uuid of Receipts as id of rewardslist. 
rewardslist = Receipts['rewardsReceiptItemList'].explode().apply(pd.Series)
rewardslist = Receipts[['_id','dateScanned','totalSpent']].join(rewardslist)

In [299]:
rewardslist

,_id,dateScanned,totalSpent,barcode,description,finalPrice,itemPrice,needsFetchReview,partnerItemId,preventTargetGapPoints,...,originalMetaBriteQuantityPurchased,pointsEarned,targetPrice,competitiveProduct,originalFinalPrice,originalMetaBriteItemPrice,deleted,priceAfterCoupon,0,metabriteCampaignId
0,5ff1e1eb0a720f0523000575,1609687531000,26.00,4011,ITEM NOT FOUND,26.00,26.00,False,1,True,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,5ff1e1bb0a720f052300056b,1609687483000,11.00,4011,ITEM NOT FOUND,1,1,NaN,1,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,5ff1e1bb0a720f052300056b,1609687483000,11.00,028400642255,DORITOS TORTILLA CHIP SPICY SWEET CHILI REDUCE...,10.00,10.00,True,2,True,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,5ff1e1f10a720f052300057a,1609687537000,10.00,NaN,NaN,NaN,NaN,False,1,True,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,5ff1e1ee0a7214ada100056f,1609687534000,28.00,4011,ITEM NOT FOUND,28.00,28.00,False,1,True,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1115,603d0b710a720fde1000042a,1614613361873,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1116,603cf5290a720fde10000413,1614607657664,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1117,603ce7100a7217c72c000405,1614604048000,34.96,B076FJ92M4,mueller austria hypergrind precision electric ...,22.97,22.97,NaN,0,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,22.97,NaN,NaN
1117,603ce7100a7217c72c000405,1614604048000,34.96,B07BRRLSVC,thindust summer face mask - sun protection nec...,11.99,11.99,NaN,1,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,11.99,NaN,NaN


In [288]:
rewardslist.columns

Index([                               '_id',
                              'dateScanned',
                                  'barcode',
                              'description',
                               'finalPrice',
                                'itemPrice',
                         'needsFetchReview',
                            'partnerItemId',
                   'preventTargetGapPoints',
                        'quantityPurchased',
                       'userFlaggedBarcode',
                       'userFlaggedNewItem',
                         'userFlaggedPrice',
                      'userFlaggedQuantity',
                   'needsFetchReviewReason',
                   'pointsNotAwardedReason',
                            'pointsPayerId',
                             'rewardsGroup',
                  'rewardsProductPartnerId',
                   'userFlaggedDescription',
                 'originalMetaBriteBarcode',
             'originalMetaBriteDescription',
          

# rename columns to fit schemas

In [220]:
Receipts.rename(columns={'_id.$oid': '_id','createDate.$date': 'createDate', 'dateScanned.$date': 'dateScanned', 
                  'finishedDate.$date':'finishedDate', 'modifyDate.$date':'modifyDate', 
                  'modifyDate.$date': 'modifyDate', 'pointsAwardedDate.$date':'pointsAwardedDate',
                  'purchaseDate.$date': 'purchaseDate'}, inplace=True)
Brand.rename(columns = {'_id.$oid':'_id'}, inplace=True)
Users.rename(columns = {'_id.$oid':'_id', 'createdDate.$date':'createdDate', 'lastLogin.$date':'lastLogin'}, 
             inplace=True)

In [110]:
desired_order = ['_id', 'bonusPointsEarned', 'bonusPointsEarnedReason', 'createDate', 'dateScanned','finishedDate',
                'modifyDate','pointsAwardedDate','pointsEarned','purchaseDate','purchasedItemCount',
                 'rewardsReceiptItemList', 'rewardsReceiptStatus', 'totalSpent', 'userId']
Receipts = Receipts.reindex(columns=desired_order)

# check duplicates

In [201]:
# Brand and Receipts have uuid, so each row should be unique.
if Brand.shape[0] == Brand.drop_duplicates().shape[0]:
    print('There is no duplicates in Brand table.')
else:
    print('There are',Brand.shape[0] - Brand.drop_duplicates().shape[0],'duplicates in Brand table.')

There is no duplicates in Brand table.


In [202]:
if Users.shape[0] == Users.drop_duplicates().shape[0]:
    print('There is no duplicates in Users table.')
else:
    print('There are',Users.shape[0] - Users.drop_duplicates().shape[0],'duplicates in Users table.')

There are 283 duplicates in Users table.


In [302]:
Users = Users.drop_duplicates()

# check missing values

In [249]:
# rate of missing values
Receipts.isnull().sum()/Receipts.shape[0]*100
# half bonusPointsEarned, pointsEarned, finishedDate, pointsAwardedDate, purchaseDate are missing.

bonusPointsEarned          51.385165
bonusPointsEarnedReason    51.385165
pointsEarned               45.576408
purchasedItemCount         43.252904
rewardsReceiptItemList     39.320822
rewardsReceiptStatus        0.000000
totalSpent                 38.873995
userId                      0.000000
_id                         0.000000
createDate                  0.000000
dateScanned                 0.000000
finishedDate               49.240393
modifyDate                  0.000000
pointsAwardedDate          52.010724
purchaseDate               40.035746
dtype: float64

In [280]:
# All receipts in 2021-03 don't have finished Date. But even in previous months, lots of missing values of date exist.
Receipts[Receipts.finishedDate.isnull()==1].YearMonth.value_counts()/Receipts.YearMonth.value_counts()

2020-10    0.500000
2020-11    0.500000
2021-01    0.241758
2021-02    0.817568
2021-03    1.000000
Freq: M, Name: YearMonth, dtype: float64

In [277]:
Receipts[Receipts.finishedDate.isnull()==1].rewardsReceiptStatus.value_counts()

SUBMITTED    434
REJECTED      71
FLAGGED       46
Name: rewardsReceiptStatus, dtype: int64

In [274]:
# rate of missing values
Brand.isnull().sum()/Brand.shape[0]*100
# categoryCode and topBrand indicator have over 50% missing values.

barcode          0.000000
category        13.281919
categoryCode    55.698372
name             0.000000
topBrand        52.442159
_id              0.000000
cpg.$id.$oid     0.000000
cpg.$ref         0.000000
brandCode       20.051414
dtype: float64

In [281]:
# rate of missing values
Users.isnull().sum()/Users.shape[0]*100

active           0.000000
role             0.000000
signUpSource     9.696970
state           11.313131
_id              0.000000
createdDate      0.000000
lastLogin       12.525253
dtype: float64

# EDA

In [272]:
Receipts.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1119 entries, 0 to 1118
Data columns (total 15 columns):
 #   Column                   Non-Null Count  Dtype  
---  ------                   --------------  -----  
 0   bonusPointsEarned        544 non-null    float64
 1   bonusPointsEarnedReason  544 non-null    object 
 2   pointsEarned             609 non-null    float64
 3   purchasedItemCount       635 non-null    float64
 4   rewardsReceiptItemList   679 non-null    object 
 5   rewardsReceiptStatus     1119 non-null   object 
 6   totalSpent               684 non-null    float64
 7   userId                   1119 non-null   object 
 8   _id                      1119 non-null   object 
 9   createDate               1119 non-null   int64  
 10  dateScanned              1119 non-null   int64  
 11  finishedDate             568 non-null    float64
 12  modifyDate               1119 non-null   int64  
 13  pointsAwardedDate        537 non-null    float64
 14  purchaseDate            

In [271]:
Receipts['pointsEarned'] = Receipts['pointsEarned'].astype('float')
Receipts['totalSpent'] = Receipts['totalSpent'].astype('float')

In [ ]:
# bonusPointsEarned column has 500+ missing values, and has no 0 value. 
# Does 'NaN' mean 0 bonus point was awarded upon receipt completion, or unknown?
# rewardsReceiptStatus=='SUBMITTED'/'PENDING', bonusPointsEarned(awarded upon receipt completion) is unknown. 
# When rewardsReceiptStatus=='FINISHED', does 'NaN' mean 0 bonus point?
# What does it mean by rewardsReceiptStatus=='FLAGGED'?
# pointsEarned column has the same situation.

In [236]:
Receipts.rewardsReceiptStatus.value_counts()

FINISHED     518
SUBMITTED    434
REJECTED      71
PENDING       50
FLAGGED       46
Name: rewardsReceiptStatus, dtype: int64

In [239]:
Receipts[(Receipts.bonusPointsEarned.isnull()==True) & (Receipts.rewardsReceiptStatus=='FINISHED')]

,bonusPointsEarned,bonusPointsEarnedReason,pointsEarned,purchasedItemCount,rewardsReceiptItemList,rewardsReceiptStatus,totalSpent,userId,_id,createDate,dateScanned,finishedDate,modifyDate,pointsAwardedDate,purchaseDate
32,NaN,NaN,500.0,9.0,"[{'barcode': '029000079236', 'description': 'P...",FINISHED,89.91,5ff36be7135e7011bcb856d3,5ff36c750a7214ada100058f,1609788533000,1609788533000,1.609789e+12,1609788534000,1.609789e+12,1.609702e+12
35,NaN,NaN,250.0,5.0,"[{'barcode': '044700009888', 'description': 'O...",FINISHED,49.95,5ff36a3862fde912123a4460,5ff36adb0a720f0523000590,1609788123000,1609788123000,1.609788e+12,1609788124000,1.609788e+12,1.609308e+12
52,NaN,NaN,350.0,1.0,"[{'barcode': '044700019917', 'description': 'O...",FINISHED,10.00,5ff370c562fde912123a5e0e,5ff3713c0a7214ada10005b6,1609789756000,1609789756000,1.609790e+12,1609789756000,1.609790e+12,1.609703e+12
70,NaN,NaN,250.0,5.0,"[{'barcode': '021000068364', 'description': 'J...",FINISHED,49.95,5ff47392c3d63511e2a47881,5ff473b10a7214ada10005c4,1609855921000,1609855921000,1.609856e+12,1609855921000,1.609856e+12,1.609567e+12
76,NaN,NaN,250.0,5.0,"[{'barcode': '044700030479', 'description': 'O...",FINISHED,49.95,5ff47392c3d63511e2a47881,5ff473f60a7214ada10005ce,1609855990000,1609855990000,1.609856e+12,1609855991000,1.609856e+12,1.609394e+12
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
641,NaN,NaN,209.9,1.0,"[{'barcode': '036000320893', 'description': 'H...",FINISHED,20.99,60147d2ac8b50e11d8453f53,601485340a720f05f8000167,1611957556000,1611957556000,1.611958e+12,1612200662000,1.611958e+12,1.611878e+12
643,NaN,NaN,209.8,1.0,"[{'barcode': '036000320893', 'description': 'H...",FINISHED,20.98,60147d2ac8b50e11d8453f53,601485370a720f05f8000172,1611957559000,1611957559000,1.611958e+12,1612200665000,1.611958e+12,1.611878e+12
664,NaN,NaN,210.0,1.0,"[{'barcode': '036000320893', 'description': 'H...",FINISHED,21.00,60147d2ac8b50e11d8453f53,601485370a720f05f8000175,1611957559000,1611957559000,1.611958e+12,1612201629000,1.611958e+12,1.611878e+12
667,NaN,NaN,209.5,1.0,"[{'barcode': '036000320893', 'description': 'H...",FINISHED,20.95,60147d2ac8b50e11d8453f53,6014853a0a7214ad50000106,1611957562000,1611957562000,1.611958e+12,1612201634000,1.611958e+12,1.611878e+12


In [273]:
# missing data in 2020-12
Receipts['YearMonth'] = pd.to_datetime(Receipts['dateScanned'], unit='ms').dt.to_period('M')
Receipts.YearMonth.value_counts()

2021-01    637
2021-02    444
2021-03     30
2020-11      6
2020-10      2
Freq: M, Name: YearMonth, dtype: int64

In [283]:
# Join Receipts and Brand on their common column 'barcode'. 
rnb = pd.merge(rewardslist[['_id','barcode','dateScanned']],Brand[['_id','name','barcode']], 
         on='barcode', how='inner')

rnb['YearMonth'] = pd.to_datetime(rnb['dateScanned'], unit='ms').dt.to_period('M')
rnb

,_id_x,barcode,dateScanned,_id_y,name,YearMonth
0,600206000a720f05f3000087,511111204206,1610745344000,5a8c36dbe4b0ccf165fac9e9,Swanson,2021-01
1,600208270a720f05f3000088,511111204206,1610745895000,5a8c36dbe4b0ccf165fac9e9,Swanson,2021-01
2,600373700a720f05f3000091,511111204206,1610838896000,5a8c36dbe4b0ccf165fac9e9,Swanson,2021-01
3,60023e8f0a720f05f300008b,511111204206,1610759823000,5a8c36dbe4b0ccf165fac9e9,Swanson,2021-01
4,60020d2a0a720f05f300008a,511111204206,1610747178000,5a8c36dbe4b0ccf165fac9e9,Swanson,2021-01
...,...,...,...,...,...,...
84,600f39c30a7214ada2000030,511111904175,1611610563000,5a8c35dde4b0ccf165fac9e6,Pepperidge Farm,2021-01
85,600f39c30a7214ada2000030,511111904175,1611610563000,5a8c35dde4b0ccf165fac9e6,Pepperidge Farm,2021-01
86,600f2fc80a720f0535000030,511111904175,1611608008000,5a8c35dde4b0ccf165fac9e6,Pepperidge Farm,2021-01
87,600f2fc80a720f0535000030,511111904175,1611608008000,5a8c35dde4b0ccf165fac9e6,Pepperidge Farm,2021-01


In [285]:
# After joining the two tables, only 2021-01 is left, 
# which means lots of barcodes in the receipts don't have corresponding brands in the Brand table.
rnb.YearMonth.value_counts()

2021-01    89
Freq: M, Name: YearMonth, dtype: int64

In [290]:
# all barcodes in Brand table start with 5xxxxx, while in Receipt table, lots of barcodes start with Bxxxx.
temp = rewardslist[rewardslist.barcode.isnull()==0]
temp[temp.barcode.str.contains('B%{,}', regex=True)]

,_id,dateScanned,barcode,description,finalPrice,itemPrice,needsFetchReview,partnerItemId,preventTargetGapPoints,quantityPurchased,...,originalMetaBriteQuantityPurchased,pointsEarned,targetPrice,competitiveProduct,originalFinalPrice,originalMetaBriteItemPrice,deleted,priceAfterCoupon,0,metabriteCampaignId
999,603983e20a7217c72c000138,1614382050000,B076FJ92M4,mueller austria hypergrind precision electric ...,22.97,22.97,NaN,0,NaN,1.0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,22.97,NaN,NaN
999,603983e20a7217c72c000138,1614382050000,B07BRRLSVC,thindust summer face mask - sun protection nec...,11.99,11.99,NaN,1,NaN,1.0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,11.99,NaN,NaN
1009,603973f80a720fde100000e3,1614377976000,B076FJ92M4,mueller austria hypergrind precision electric ...,22.97,22.97,NaN,0,NaN,1.0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,22.97,NaN,NaN
1009,603973f80a720fde100000e3,1614377976000,B07BRRLSVC,thindust summer face mask - sun protection nec...,11.99,11.99,NaN,1,NaN,1.0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,11.99,NaN,NaN
1015,603978600a7217c72c00010b,1614379104000,B08BGBHHP6,spigen thin fit designed for iphone 12 mini ca...,12.59,12.59,NaN,0,NaN,1.0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,12.59,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1113,603cc2bc0a720fde100003e9,1614594748000,B07BRRLSVC,thindust summer face mask - sun protection nec...,11.99,11.99,NaN,1,NaN,1.0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,11.99,NaN,NaN
1114,603cc0630a720fde100003e6,1614594147000,B076FJ92M4,mueller austria hypergrind precision electric ...,22.97,22.97,NaN,0,NaN,1.0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,22.97,NaN,NaN
1114,603cc0630a720fde100003e6,1614594147000,B07BRRLSVC,thindust summer face mask - sun protection nec...,11.99,11.99,NaN,1,NaN,1.0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,11.99,NaN,NaN
1117,603ce7100a7217c72c000405,1614604048000,B076FJ92M4,mueller austria hypergrind precision electric ...,22.97,22.97,NaN,0,NaN,1.0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,22.97,NaN,NaN


In [287]:
# The same barcode corresponds to different brands, which is not common.
Brand.groupby('barcode',)._id.count().sort_values()

barcode
511111000167    1
511111616535    1
511111616467    1
511111616306    1
511111616252    1
               ..
511111004790    2
511111704140    2
511111504139    2
511111305125    2
511111605058    2
Name: _id, Length: 1160, dtype: int64

In [291]:
# According to the schema, it's set to constant value 'CONSUMER'. 
Users.role.value_counts()

consumer       413
fetch-staff     82
Name: role, dtype: int64

In [ ]:
# There are more unique userIds in Receipts table than in Users table. 

In [295]:
Users._id.drop_duplicates().shape[0]

212

In [297]:
Receipts.userId.drop_duplicates().shape[0]

258